### Sentiment anlayis for the Reviews Using VENDER

 

<b> Importing needed Libaries

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install vaderSentiment
import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

     |████████████████████████████████| 133kB 5.4MB/s 


<b>Creating Data Frame with the csv file

In [3]:
filePath = '/content/drive/MyDrive/SDGP/Main/Data Science/Dataset/reviews_only(Sentiment)_cleaned.csv'
df = pd.read_csv(filePath)
df.head()

,reviewText
0,use year basically quiet environment sound pre...
1,own two years wear thirty time mostly plane ri...
2,buy think ticket noisefree pleasant air travel...
3,own years use fly two round trip week wear con...
4,love headphone get narita airport obsess whole...


<b>Using VADER to analayze the sentiment of the reviews


In [ ]:
df = df.drop(['LABEL','DOC_ID', 'RATING', 'VERIFIED_PURCHASE', 'PRODUCT_CATEGORY', 'PRODUCT_ID', 'PRODUCT_TITLE', 'REVIEW_TITLE'], axis="columns")
df

,LABEL,REVIEW_TEXT
0,Fake,"When least you think so, this product will sav..."
1,Fake,Lithium batteries are something new introduced...
2,Fake,I purchased this swing for my baby. She is 6 m...
3,Fake,I was looking for an inexpensive desk calcolat...
4,Fake,I only use it twice a week and the results are...
...,...,...
20995,Real,"I bought these for work. I have high arches, ..."
20996,Real,Crocs are one of only two brands of shoes that...
20997,Real,I love moccasins This fit like it was custom ...
20998,Real,I wish these were a little more durable. I got...


In [ ]:
df = df.drop([''], axis="columns")
df

,REVIEW_TEXT
0,"When least you think so, this product will sav..."
1,Lithium batteries are something new introduced...
2,I purchased this swing for my baby. She is 6 m...
3,I was looking for an inexpensive desk calcolat...
4,I only use it twice a week and the results are...
...,...
20995,"I bought these for work. I have high arches, ..."
20996,Crocs are one of only two brands of shoes that...
20997,I love moccasins This fit like it was custom ...
20998,I wish these were a little more durable. I got...


In [4]:
# Getting a compound score
def sentimental_compound(sentence):
    analyzer = SentimentIntensityAnalyzer()
    df_sentiment = analyzer.polarity_scores(sentence)
    score = df_sentiment['compound']
    return score

# VADER predicting Positive negative
def sentimental_score(sentence):
    analyzer = SentimentIntensityAnalyzer()
    df_sentiment = analyzer.polarity_scores(sentence)
    score = df_sentiment['compound']
    if score >= 0.5:
        return 'pos'
    elif (score > -0.5) and (score < 0.5) :
        return 'neu'
    elif score <= -0.5:
        return 'neg'
    

### Importing SVM classification

In [5]:
#!pip3 install sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline

In [6]:
# # selecting 1000 rows to avoid crashing the browser
# df_selected_rows = df.head(1000)
# #df.insert(1, "lab",0)
# df_selected_rows


In [10]:
# adding sentiment scores for each reviews
df['compound']=df['reviewText'].apply(lambda x: sentimental_score(str(x)))
df.head()


,reviewText,compound
0,use year basically quiet environment sound pre...,pos
1,own two years wear thirty time mostly plane ri...,pos
2,buy think ticket noisefree pleasant air travel...,pos
3,own years use fly two round trip week wear con...,pos
4,love headphone get narita airport obsess whole...,pos


In [11]:
df

,reviewText,compound
0,use year basically quiet environment sound pre...,pos
1,own two years wear thirty time mostly plane ri...,pos
2,buy think ticket noisefree pleasant air travel...,pos
3,own years use fly two round trip week wear con...,pos
4,love headphone get narita airport obsess whole...,pos
...,...,...
87979,please headphone overall noise cancellation be...,pos
87980,good sound great noise cancellation hope barel...,pos
87981,headphone great job dampen sound even noise ca...,pos
87982,sound wise nice enough heavy base heavy anythi...,pos


In [12]:
# Using SVM classifier on the train set
from sklearn.svm import LinearSVC
#classifer = LinearSVC(max_iter=df.shape[0])


In [13]:
tfidf_lr_pipe = Pipeline([('tfidf', TfidfVectorizer(lowercase=False)), ('svm', LinearSVC(max_iter=df.shape[0]))])

In [14]:
df.head(5)

,reviewText,compound
0,use year basically quiet environment sound pre...,pos
1,own two years wear thirty time mostly plane ri...,pos
2,buy think ticket noisefree pleasant air travel...,pos
3,own years use fly two round trip week wear con...,pos
4,love headphone get narita airport obsess whole...,pos


In [15]:
df['reviewText']=df['reviewText'].apply(str)
df['compound']=df['compound'].apply(str)

In [16]:
df.head(5)

,reviewText,compound
0,use year basically quiet environment sound pre...,pos
1,own two years wear thirty time mostly plane ri...,pos
2,buy think ticket noisefree pleasant air travel...,pos
3,own years use fly two round trip week wear con...,pos
4,love headphone get narita airport obsess whole...,pos


In [17]:
x = df.iloc[:,0].values
x

array(['use year basically quiet environment sound pretty good good similarlyprice studioclass headset pay noise cancel raw capability pretty decent acceptable detail treble vocal bass range noise cancel work pretty well take several intercontinental flight decent job block engine rumble scream kid also decent job block adult human voice nice office however must pretty big ear find foam cup quite thick enough adequate push driver away outer ear two three hour use ear start ache bite cord plenty long disconnect middle link optional sony wire remote control adapter convert standard 18th inch stereo jack dual 18th inch jack many airplane seat include thing like find uncomfortable try lean head window airplane sleep think mdrnc11s might better choice regard also kind bulky unless large carryon keep paranoid putt seat pocket front since mdrnc20s come soft bag instead rigid clamshell like bose headset spill yet active traveller may wish find case rigid pretty happy natural audio sound noise 

In [18]:
y = df.iloc[:,1].values
y

array(['pos', 'pos', 'pos', ..., 'pos', 'pos', 'neu'], dtype=object)

In [19]:
# Splitting the train and test data (80/20)

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state= 0)

In [20]:
# Converting the reviews to a matrix of token counts (Bag of words model)
tfidf_lr_pipe.fit(x_train, y_train)

Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=False, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('svm',
                 LinearSVC(C=1.0, class_weight=None, dual=True,
                           fit_intercept=True, intercept_scaling=1,
              

In [21]:
y_pred = tfidf_lr_pipe.predict(x_test)

In [22]:
predicted_df = pd.DataFrame({'predict': y_pred, 'actual': y_test})

In [23]:
predicted_df

,predict,actual
0,pos,pos
1,pos,pos
2,pos,pos
3,pos,pos
4,neu,neu
...,...,...
17592,pos,neu
17593,pos,pos
17594,pos,pos
17595,pos,pos


In [24]:
predicted_df.head(100)

,predict,actual
0,pos,pos
1,pos,pos
2,pos,pos
3,pos,pos
4,neu,neu
...,...,...
95,neu,neu
96,pos,pos
97,pos,pos
98,pos,pos


2

In [25]:
from sklearn.metrics import classification_report
# y_unique = y_test.unique()
# mcm = multilabel_confusion_matrix(y_test, y_pred, labels = y_unique)

In [26]:
from sklearn.metrics import classification_report

print('SVM results')
#target_names = [' class 0(pos)','class 1(neutral)','class 2(neg)']
print(classification_report(y_test, y_pred,  labels=["pos", "neu", "neg"]))

# Accuracy = (TP + TN) / (TP + TN + FP + FN)
# Precision = (TP) / (TP + FP)
# Recall = (TP) / (TP + FN)
# F1 Score = (2 * Precision * Recall) / (Precision + Recall)

SVM results
              precision    recall  f1-score   support

         pos       0.93      0.97      0.95     13425
         neu       0.76      0.71      0.74      3580
         neg       0.65      0.26      0.37       592

    accuracy                           0.89     17597
   macro avg       0.78      0.65      0.68     17597
weighted avg       0.88      0.89      0.88     17597



In [27]:
# Confusion matrix to see the number of true positive, true negative, false positive and false negative
from sklearn.metrics import multilabel_confusion_matrix
multilabel_confusion_matrix(y_test, y_pred, labels=["pos", "neu", "neg"])


array([[[ 3172,  1000],
        [  439, 12986]],

       [[13199,   818],
        [ 1023,  2557]],

       [[16923,    82],
        [  438,   154]]])

In [28]:
import pickle
filename = "/content/serialize_sentiment_model.pkl"  

with open(filename, 'wb') as file:  
    pickle.dump(tfidf_lr_pipe, file)